In [1]:
import os
import fastavro
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from correction.compute import *

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
data = pd.read_csv("../data_examples/csv/raw_detections.csv", index_col=False)

In [ ]:
# test c2
data.at[180, "isdiffpos"] = 'f'

In [ ]:
# test c3
data.at[150, "distnr"] = 0.1

In [ ]:
print(data.objectId.unique(), data.fid.unique())

In [ ]:
pd.options.display.max_rows = 999
display(data[["objectId", "fid", "magpsf", "distnr", "chinr", "sharpnr"]])

In [ ]:
def apply_correction_df(data):
    
    # apply correction where possible, save corrected column
    df = data.copy()
    #display(df.head(1))
    #print(df.objectId.unique(), df.fid.unique())
    df['isdiffpos'] = df['isdiffpos'].map({'t': 1., 'f': -1.})
    df["corr"] = df["distnr"] < 1.4
    correction_results = df.apply(lambda x: correction(x.magnr, x.magpsf, x.sigmagnr, x.sigmapsf, x.isdiffpos) if x["corr"] else (np.nan, np.nan, np.nan), axis=1, result_type="expand")
    df["magpsf_corr"], df["sigmapsf_corr"], df["sigmapsf_corr_ext"] = correction_results[0], correction_results[1], correction_results[2]
    
    # check for consistency
    idxmin = df.candid.idxmin()
    df["corr_magstats"] = df.loc[idxmin]["corr"]
    
    # check if suspicious behavior in the light curve (c1 | c2 | c3)
    mask = ((df["corr"] == False) & (df.isdiffpos == -1)) | (df.corr_magstats & (df["corr"] == False)) | ((df.corr_magstats == False) & df["corr"])
    df["dubious"] = mask
    
    # stellar_object
    idxmin = df.candid.idxmin()
    nearZTF = (df.loc[idxmin].distnr < 1.4) # near a ZTF object
    nearPS1 = (df.loc[idxmin].distpsnr1 < 1.4) # near a PS1 object
    stellarPS1 = (df.loc[idxmin].sgscore1 > 0.4)  # nearest object in PS1 is stelar
    stellarZTF = (df.loc[idxmin].chinr < 2) & (df.loc[idxmin].sharpnr > -0.13) & (df.loc[idxmin].sharpnr < 0.1) # nearest object in ZT is stellar
    df["stellar_object"] = (nearZTF & nearPS1 & stellarPS1) | (nearZTF & ~nearPS1 & stellarZTF)
    df["stellar_magstats"] = (nearZTF & stellarZTF)
    
    return df

In [6]:
def apply_magstats(df):
    response = {}
    idxmin = df.candid.idxmin()
    idxmax = df.candid.idxmax()
    nearZTF = (df.loc[idxmin].distnr < 1.4) # near a ZTF object
    nearPS1 = (df.loc[idxmin].distpsnr1 < 1.4) # near a PS1 object
    stellarPS1 = (df.loc[idxmin].sgscore1 > 0.4)  # nearest object in PS1 is stelar
    stellarZTF = (df.loc[idxmin].chinr < 2) & (df.loc[idxmin].sharpnr > -0.13) & (df.loc[idxmin].sharpnr < 0.1) # nearest object in ZT is stellar 
    response["ndet"] = len(df)
    response["rfid"] = df.loc[idxmin].rfid # ?
    response["nrfid"] = len(df.rfid.dropna().unique())
    response["magnr"] = df.loc[idxmin].magnr # ?
    response["stellar_object"] = (nearZTF & nearPS1 & stellarPS1) | (nearZTF & ~nearPS1 & stellarZTF)
    response["stellar_magstats"] = (nearZTF & stellarZTF)
    response["magap_mean"] = df.magap.mean()
    response["magap_median"] = df.magap.median()
    response["magap_max"] = df.magap.max()
    response["magap_min"] = df.magap.min()
    response["magap_fisrt"] = df.loc[idxmin].magap
    response["magap_last"] = df.loc[idxmax].magap
    response["first_mjd"] = df.loc[idxmin].jd - 2400000.5
    response["last_mjd"] = df.loc[idxmax].jd - 2400000.5
    return pd.Series(response)

In [ ]:
dflarge = data.groupby(["objectId", "fid"]).apply(apply_correction_df)

In [ ]:
dflarge[["objectId", "fid", "distnr", "distpsnr1", "sgscore1", "chinr", "sharpnr", "corr", "corr_magstats", "dubious", "stellar_object", "stellar_magstats"]]

In [4]:
groups = data.groupby(["objectId", "fid"])

In [8]:
magstats = groups.apply(apply_magstats)
display(magstats)

ndet         rfid  nrfid      magnr  stellar_object  \
objectId     fid                                                        
ZTF18aaiscil 1     270          NaN      2  18.725000            True   
             2     267          NaN      3  18.410000            True   
ZTF18aazxcwf 1      57  481120101.0      1  15.888000            True   
             2      56  481120201.0      1  15.716000            True   
ZTF18abdgukn 1      55  720120155.0      1  17.815001           False   
             2      42  720120255.0      2  17.340000           False   
ZTF19aazzpje 1       9          NaN      1  19.525000           False   
             2      10  632120231.0      1  18.596001           False   
ZTF19abaejrh 1       5          NaN      1  18.702999           False   
             2       8  599120201.0      1  18.254000           False   
ZTF20aaelulu 1      31  576120110.0      1  16.099001           False   
             2      27  576120210.0      1  15.832000           False   

                  stellar_magstats  magap_mean  magap_median  magap_max  \
objectId     fid                                                          
ZTF18aaiscil 1                True   18.216508     17.764299  19.830000   
             2                True   18.398000     18.400200  19.263100   
ZTF18aazxcwf 1                True   17.123884     17.018200  18.760500   
             2                True   17.256650     17.174900  19.217300   
ZTF18abdgukn 1               False   19.671209     19.684200  20.398800   
             2               False   19.792521     19.787250  20.466499   
ZTF19aazzpje 1               False   19.258200     19.066900  20.118999   
             2               False   19.394120     19.293350  20.365499   
ZTF19abaejrh 1               False   19.071700     19.156099  19.575300   
             2               False   19.300600     19.175150  19.866199   
ZTF20aaelulu 1               False   16.860161     16.848499  18.524000   
             2               False   16.245715     16.440100  18.117599   

                  magap_min  magap_fisrt  magap_last     first_mjd  \
objectId     fid                                                     
ZTF18aaiscil 1    17.142300    19.830000   18.880800  58248.194780   
             2    18.034100    19.134899   18.276699  58242.264363   
ZTF18aazxcwf 1    16.140100    17.043600   16.208000  58346.147766   
             2    16.453800    16.877899   17.323000  58281.209780   
ZTF18abdgukn 1    18.893299    19.791000   20.178499  58300.200081   
             2    19.290501    19.884899   19.338499  58306.194479   
ZTF19aazzpje 1    18.712700    20.118999   19.832800  58647.215463   
             2    18.830200    19.207500   20.365499  58653.207245   
ZTF19abaejrh 1    18.532499    18.532499   19.575300  58653.380359   
             2    18.797701    19.767700   19.590000  58653.443403   
ZTF20aaelulu 1    14.328200    14.956700   18.524000  58859.522280   
             2    13.872700    17.158300   18.117599  58855.542292   

                      last_mjd  
objectId     fid                
ZTF18aaiscil 1    59004.198796  
             2    59004.255984  
ZTF18aazxcwf 1    59005.228102  
             2    59001.259282  
ZTF18abdgukn 1    58875.485081  
             2    58765.112974  
ZTF19aazzpje 1    58675.298102  
             2    58684.187349  
ZTF19abaejrh 1    58675.447743  
             2    58688.408079  
ZTF20aaelulu 1    58967.227049  
             2    58965.286690